Below is the search algorithm taken from the full experiment. 

L = list of symbols

R_M = list prerequisite equations

KB = L + R_M

In [2]:
def predict_intermediate_state(first, last, epsilon, n_diff_syms, n_part_of_math, letter):

    n1 = n_diff_syms
    n2 = n_part_of_math

    paths = []
    path_actions = []
    state = first.args

    Self_SA = self_state_actions(None, state)
    for name in variable_names(Self_SA):
        s = getattr(Self_SA, name)
        if s == None:
            pass
        else:
            try:
                middle = sp.Eq(s[0], s[1])
                syms = list(middle.args[0].atoms(sp.Symbol)) + list(middle.args[1].atoms(sp.Symbol))
                if (first, middle, last) not in paths and sp.Symbol('?') not in syms:
                    paths.append((first, middle, last)) 
                    path_actions.append([name])
            except:
                pass

    L = L_M + [i for i in find_atoms(last.args) if i not in find_atoms(first.args)] + [i for i in find_atoms(first.args) if i not in find_atoms(last.args)]
    L = list(dict.fromkeys(L))

    for symbol in L:
        Symbol_SA = symbol_state_actions(symbol, state)
        for name in variable_names(Symbol_SA):
            s = getattr(Symbol_SA, name)
            if s == None:
                pass
            else:
                try:
                    middle = sp.Eq(s[0], s[1])
                    syms = list(middle.args[0].atoms(sp.Symbol)) + list(middle.args[1].atoms(sp.Symbol))
                    if (first, middle, last) not in paths and sp.Symbol('?') not in syms:
                        paths.append((first, middle, last))
                        path_actions.append([name])
                except:
                    pass

    for equation in R_M:
        Equation_SA = equation_state_actions(equation, state)
        for name in variable_names(Equation_SA):
            s = getattr(Equation_SA, name)
            if s == None:
                pass
            else:
                try:
                    middle = sp.Eq(s[0], s[1])
                    syms = list(middle.args[0].atoms(sp.Symbol)) + list(middle.args[1].atoms(sp.Symbol))
                    if (first, middle, last) not in paths and sp.Symbol('?') not in syms:
                        paths.append((first, middle, last))
                        path_actions.append([name])
                except:
                    pass

    print("total number of paths = %d"%(len(paths)))


    path_H_pairs = []
    H_list = []
    for path in paths:
        path_H_pairs.append((path, H(path[1], path[2], True, n1, n2)))
        H_list.append(H(path[1], path[2], False, n1, n2))

    H_list = sorted(H_list)

    new_paths = []
    new_path_actions = []
    for i in range(len(H_list)):
        for j in range(len(path_H_pairs)):
            if path_H_pairs[j][1][0] == H_list[i] and path_H_pairs[j] not in new_paths:
                new_paths.append(path_H_pairs[j])
                new_path_actions.append(path_actions[j])

    s_hat_i_candidates = []
    total_distances = []
    s_hat_i_candidates_actions = []

    for i in range(len(new_paths)):
        print('path = %d'%(i)) if i%5 == 0 else 0
        path = new_paths[i]
        action = new_path_actions[i]
        state = path[0][1].args
        state_history.append(state)

        final_states = []
        final_actions = []

        Self_SA = self_state_actions(None, state)
        for name in variable_names(Self_SA):
            s = getattr(Self_SA, name)
            if s == None:
                pass
            else:
                try:
                    final_state = sp.Eq(s[0], s[1])
                    if final_state not in final_states:
                        final_states.append(final_state)
                        final_actions.append([name])
                except:
                    pass

        for symbol in L:
            Symbol_SA = symbol_state_actions(symbol, state)
            for name in variable_names(Symbol_SA):
                s = getattr(Symbol_SA, name)
                if s == None:
                    pass
                else:
                    try:
                        final_state = sp.Eq(s[0], s[1])
                        if final_state not in final_states:
                            final_states.append(final_state)
                            final_actions.append([name])
                    except:
                        pass

        for equation in R_M:
            Equation_SA = equation_state_actions(equation, state)
            for name in variable_names(Equation_SA):
                s = getattr(Equation_SA, name)
                if s == None:
                    pass
                else:
                    try:
                        final_state = sp.Eq(s[0], s[1])
                        if final_state not in final_states:
                            final_states.append(final_state)
                            final_actions.append([name])
                    except:
                        pass

        distances = [M(final_state, last, letter) for final_state in final_states]
        index = np.argmin(distances)

        state_history.remove(state)

        if distances[index] == 0:
            return path[0][1], action + final_actions[index]

        elif distances[index] <= epsilon:
            s_hat_i_candidates.append(path[0][1])
            s_hat_i_candidates_actions.append([final_actions[index]])
            total_distances.append(distances[index])


    print('\n')
    if total_distances == []:
        return sp.Eq(sp.Symbol('x'), sp.Symbol('?')), ['N/A', 'N/A'] 
    else:
        index = np.argmin(total_distances)
        if s_hat_i_candidates == []:
            return sp.Eq(sp.Symbol('x'), sp.Symbol('??')), ['N/A', 'N/A'] 
        else:
            return s_hat_i_candidates[index], action + s_hat_i_candidates_actions[index]